# **01. PyTorch - Multi-Layer Perceptron**


## [Step 01] Dataset & DataLoader

- **Dataset** : Dataset 클래스를 상속 --> Custom Instance를 생성

- **DataLoader**: 학습 시 각 Instance에 쉽게 접근하기 위해 순회 가능한 객체(iterable) 생성, shuffling 및 batch 등의 작업 수행

<p align="center">
<img src = https://sebastianraschka.com/images/blog/2022/datapipes/loader-flow.png width = 500>
</p>

<p align="center">
<img src = https://sebastianraschka.com/images/blog/2022/datapipes/dataflow-good.png width = 350>
</p>

- 예제 코드
```
from torch.utils.data import  TensorDataset, DataLoader
```
```
# X,y로 분할한 데이터를 tensor로 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)
```
```
# tensor를 TensorDataset으로 생성 - X와 y가 짝으로 이루어짐
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
```
```
# DataLoader 형태로 생성
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)
```

## [Step 02] 디바이스 설정

- 일반적으로 인공신경망의 학습은 (가능하다면) GPU를 사용하는 것이 바람직함
    - Colab Runtime 설정 변경
- GPU를 사용하여 학습을 진행하도록 명시적으로 작성 필요
- 연산 유형에 따라 GPU에서 수행이 불가능한 경우도 존재하는데, 그럴 경우도 마찬가지로 명시적으로 어떤 프로세서에서 연산을 수행해야하는지 코드로 작성해야함 

```
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = NeuralNetwork().to(device)
```


## [Step 03] 신경망 모델 생성

- **torch.nn 패키지**는 신경망 생성 및 학습 시 설정해야하는 다양한 기능을 제공

```
import torch.nn as nn
```
- 신경망을 **nn.Module**을 상속받아 정의
    - __ __init__ __(): 신경망에서 사용할 layer를 초기화하는 부분
    - __forward()__: feed foward 연산 수행 시, 각 layer의 입출력이 어떻게 연결되는지를 지정

```
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.input_layer    = nn.Linear(4, 16)
        self.hidden_layer1  = nn.Linear(16, 32)
        self.output_layer   = nn.Linear(32, 3)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out

```


## [Step 04] 모델 컴파일(Compiling)

- 학습 시 필요한 정보들(loss function, optimizer)을 선언
- 일반적으로 loss와 optimizer는 아래와 같이 변수로 선언하고, 변수를 train/test 시 참고할 수 있도록 매개변수로 지정해줌 

```
learning_rate = 0.01
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
```


## [Step 05] 모델 학습(Training)

- **신경망의 학습과정**을 별도의 함수로 구성하는 것이 일반적
    - feed forward -> loss -> error back propagation -> (log) -> (반복)

```
def train_loop(train_loader, model, loss_fn, optimizer):
    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
```

## [Step 06] 학습된 모델 테스트

- 학습과정과 비슷하나 error back propagate하는 부분이 없음
    - feed forward -> loss ->  (log) -> (반복)

```
def test_loop(test_loader, model, loss_fn):
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f}\n")


## [Step 07] Iteration
- 신경망 학습은 여러 epochs을 반복해서 수행하면서 모델을 구성하는 최적의 파라미터를 찾음
- 지정한 epochs 수만큼 **학습**과정과 **평가**과정을 반복하면서, 모델의 성능(loss, accuracy 등)을 체크함

```
epochs = 10
for i in range(epochs) :
    print(f"Epoch {i+1} \n------------------------")
    train_loop(train_dataloader, model, loss, optimizer)
    test_loop(test_dataloader, model, loss)
print("Done!")
```

# **02. MLP - IRIS 데이터셋**

- 데이터명: IRIS (아이리스, 붗꽃 데이터)  
- 데이터설명: 꽃잎의 각 부분의 너비와 길이등을 측정한 데이터
- 레코드수: 150개
- 필드개수: 5개(총 6개의 필드로 구성, caseno는 순서를 표시하므로 분석에서 제외), 2-5번째의 필드는 입력 변수로 사용, 맨 아래의 Species 속성은 종속변수
  - caseno: 일련번호(1부터 150까지 입력)
  - Sepal Length:	꽃받침 길이
  - Sepal Width:	꽃받침 너비
  - Petal Length:	꽃잎 길이
  - Petal Width:	꽃잎 너비  
  - Species: 꽃의 종류(setosa / versicolor / virginica)

## [Step 01] 라이브러리 및 데이터셋 설정



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import  TensorDataset, DataLoader

# 데이터 불러오기
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

In [2]:
print(df.shape)
df.describe()

(150, 4)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [3]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [ ]:
df['label'] = iris.target

# 데이터 분할(학습용 및 테스트용 데이터)
y = df['label']
X = df.drop(['label'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state=42, stratify=y)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(112, 4) (112,)
(38, 4) (38,)


## [Step 02] DataLoader 생성

In [ ]:
# X_train: [112,4], X_test: [38,4]
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

## [Step 03] 모델 구성

In [ ]:
# 모든 네트워크는 기본 클래스 nn.Module으로부터 상속받아 정의
class NeuralNetwork(nn.Module):

    # 사용하고자 하는 레이어 정의
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.input_layer    = nn.Linear(4, 16)
        self.hidden_layer1  = nn.Linear(16, 32)
        self.output_layer   = nn.Linear(32, 3)
        self.relu = nn.ReLU()
    
    # 정의된 레이어를 통한 모델 정의
    def forward(self,x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out

## [Step 04] 모델 인스턴스 생성

In [ ]:
# GPU 자원 사용 가능확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device = {device}')

model = NeuralNetwork().to(device)

device = cuda


## [Step 05] 모델 컴파일

In [ ]:
# 모델 컴파일
learning_rate = 0.001
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## [Step 06] 모델 학습 함수

In [ ]:
def train_loop(train_loader, model, loss_fn, optimizer):
    size = len(train_loader.dataset)

    for batch, (X, y) in enumerate(train_loader):

        # X, y 텐서를 device로 전송
        X, y = X.to(device), y.to(device)

        # 백그라운드로 forward를 실행
        pred = model(X)

        # 손실 계산
        loss = loss_fn(pred, y)

        optimizer.zero_grad() # Gradient를 0으로 셋팅, Gradient가 누적되는 것을 방지
        loss.backward()       # Backpropagation
        optimizer.step()      # 역전파 단계에서 수집된 변화도로 매개변수를 업데이트
        
        loss, current = loss.item(), batch * len(X)
        print(f'loss: {loss:>7f}  [{current:>5d}]/{size:5d}')

## [Step 07] 모델 테스트 함수

In [ ]:
def test_loop(test_loader, model, loss_fn):
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    # Gradient를 계산하는 autograd를 비활성화
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)

            # 평균 손실을 구하기 위해 손실 누적 값 계산
            test_loss += loss_fn(pred, y).item()

            # 정확도 누적값 계산
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f}\n")

## [Step 08] 모델 학습

In [ ]:
# 모델 실행
epochs = 100

for i in range(epochs) :
    print(f"Epoch {i+1} \n------------------------")
    train_loop(train_dataloader, model, loss, optimizer)
    test_loop(test_dataloader, model, loss)
print("Done!")